In [133]:
import random
# from abc import ABCMeta, abstractmethod

class CardShoe(object):
    '''
    This abstract base class is used to simulate a six deck shoe.
        
    Class Order Attributes:
        suits: a list of the suits used in normal playing card decks
            S (spades), D (diamonds), H (hearts), and C (clubs). ['S', 'D',
            'H', 'C']
        ranks: a list of the ranks of playing cards in ascending order Ace 
            through King, represented by A, 1, 2, 3,...., 9, 10, J, Q, K.
            ['A', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 
            'Q', 'K']
    
    Attributes
        shuffled_deck : a randomly shuffled shoe created from decks.
        length: the number of cards in the shoe after initialization
    
    Methods:
        __init__ : Initializes shuffled_deck to create a card shoe.
        __str__: prints out the number of cards remaining in the shoe.
        __del__: prints a message the deck show has been removed as it 
            deletes the CardShoe object
        remove_top: removes the top card (index 0) from the shuffled deck
            and returns the tuple of the card (rank, suit)
        
    '''
    suits = ['S', 'D', 'H', 'C']
    ranks = ['A', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K']
    
    def __init__(self):
        # decks is created as a set of indexes used to create a shuffled
        # six deck shoe. It is a local variable, not an attribute. It is
        # an ordered list of tuples of the form (rank, suit), where rank
        # is taken from the list CardShoe.rank and suit from list 
        # CardShoe.suits.
        # To prevent cheating, __init__ takes no arguments. Instead,
        # it creates an empty shuffled_deck itself.
        decks = []
        for i in xrange(0, 6):
            for s in CardShoe.suits:
                for r in CardShoe.ranks:
                    decks.append((r,s))
        
        # To create a shuffled card shoe, we need to take a random card from
        # decks and move it into the shoe. We keep doing that until the deck
        # is fully randomized.
        self.shuffled_deck = []
        while len(decks) > 0:
            # card is another local variable. It is a tuple pulled using a
            # random index of the remaining elements in decks.
            # card_index is that random index created using random.randint().
            card_index = random.randint(0,len(decks) - 1)
            self.shuffled_deck.append(decks[card_index])
            del decks[card_index]
        # Now, CardShoe.shuffled_deck contains a shuffled six deck shoe.
        # This length is the number of cards in the initialized deck.
        self.length = len(self.shuffled_deck)
    
    def __len__(self):
        # This len method will print out the number of cards left in the shoe.
        return len(self.shuffled_deck)
    
    def __str__(self):
        # This prints out a statement indicating that the deck has been
        # initialized and the current entries remaining in it.
        return "A shuffled shoe containing "+ str(len(self.shuffled_deck)) + " cards."
    
    def __del__(self):
        print("The current deck shoe has been removed from the game.")
        return True
    
    def remove_top(self):
        '''
        This method removes the top card from the deck and returns the tuple
        (rank, suit) of the card. The top card has index 0. It does not accept
        any arguments.
        '''
        top_card = self.shuffled_deck[0]
        self.shuffled_deck.pop(0)
        return top_card

In [134]:
x = CardShoe()

The current deck shoe has been removed from the game.


In [135]:
print(x)

A shuffled shoe containing 312 cards.


In [136]:
from __future__ import print_function

class Player(object):
    """
    This class creates Hands for a Blackjack player.
    
    Terms related to game play:
        players_turn: The part of a round of play in which players are playing their hands, in
            rotation around the "table"
        dealer_turn: The part of the round of play in which the dealer plays his/her hand
        round: A deal, followed by players deciding bets and card draws, followed by dealer's turn,
            and ending with settling wins and losses for the player hands
        game: A series of rounds of play until either all players run out of money or decide to
            stop playing and "cash out". After all players quit, a final tally of their money
            determines the winner of the game.
    
    Class Order Attributes:
        values: dictionary of the scoring for each rank of blackjack card (it was not in use
            in CardShoe class). It is a COA because it is needed for every player object.
        
    Attributes:
        name: stores the player's name
        hand: tracks the card tuples (rank, suit) of the cards in the player's regular hand
        soft_hand_score: integer value of the current "soft" score of the player's hand
        hard_hand_score: integer value of the "hard" score of the player's hand
        split_flag: boolean set to True if the player splits, False if not.
        split_hand: tracks a second hand created by a split
        soft_split_score: integer value of the current "soft" score of the player's split hand,
            if it exists
        hard_split_score: integer value of the current "hard" score of the player's split hand,
            if it exists
        bank: integer number of dollars the player currently has in chips
        bet: current amount bet on the outcome of the hand vs dealer's hand
        split_bet: tracks any bets applied to a card split hand
        insurance: tracks the amount of any side bets taken on the dealer getting blackjack
    
    Methods:
        __init__: Creates the player object, initializing all of the attributes. The bank has a
            default value of 1000 dollars.
        __str__: This method prints the player's name, the player's bank, bets, and primary hand.
            If the split_flag is True, it will print that data as well.
        __del__: prints a message while removing the player object
        __len__: prints out the len of the player's regular hand (This is used to determine
            player blackjack)
        print_split: This method does the same thing as __str__ except that it prints only the data
            for the split hand
        add_card_to_hand: takes a card tuple as an argument and adds it to the hand and updates the
            soft and hard scores for Player.hand. If the both scores exceed 21, return False.
            Otherwise, return True.
        add_card_to_split: performs the same operation on the split hand if one exists
        blackjack: takes the player's regular bet, mulitplies it by the Blackjack multiplier
            (supplied via argument), and clears the regular hand attributes
        win: adds the player's bet to the bank and clears the regular hand attributes
        split_win: add the player's split bet to the bank and clears the split hand attributes
        loss: subtracts the player's bet from the bank and clears the regular hand
            attributes
        split_loss: subtracts the player's split bet from the bank and clears the split hand
            attributes
        tie: clears the player's regular hand attributes without altering the bank
        split_tie: clears the player's regular hand attributes without altering the bank
        split_pair: moves one card over to the split_hand, prompts for a initial
            split_bet, and sets the split_flag to True. Adjusts the scores accordingly.
        start_round: 
        start_game: 
        end_round: verifies that all hands are empty, the split_flag has been reset, and all bets
            have been reset to zero (including insurance)
      
    """
    values = {'A': 1, '2' : 2, '3' : 3, '4' : 4, '5' : 5, '6' : 6, \
              '7' : 7, '8' : 8, '9' : 9, '10' : 10, 'J' : 10, 'Q' : 10,\
              'K' : 10 }
    
    
    def __init__(self, name, bank=1000):
        '''
        This method intializes all of the following attributes:
            name: takes a string to initialize the player's name
            bank: takes a non-negative integer and stores it (even for Dealer). This attribute
                has a default of $1000 dollars if not specified
            hand: creates an empty list
            soft_hand_score: integer set to 0
            hard_hand_score: integer set to 0
            split_flag: boolean set to False
            split_hand: creates an empty list
            soft_split_score: integer set to 0
            hard_split_score: integer set to 0
            bet: integer set to 0
            split_bet: integer set to 0
            insurance: integer set to 0
            
        '''
        self.name = name
        self.bank = bank
        self.hand = []
        self.soft_hand_score = 0
        self.hard_hand_score = 0
        self.split_flag = False
        self.split_hand = []
        self.soft_split_score = 0
        self.hard_split_score = 0
        self.bet = 0
        self.split_bet = 0
        self.insurance = 0
    
    def __str__(self):
        '''
        This method prints out the full data on a Player in a readable format. If
        the split_flag is True, it adds the data for a split hand as well.
        '''
        print("Player:\t", self.name)
        print("Chips:\t${0}.00".format(self.bank))
        print("\n\tCurrent Hand: ", end='')
        # This suppresses the linefeed and flushes the buffer to make the ouput
        # look like a single line of code.
                                                      
        for rank, suit in self.hand:
            print("{0}-{1}  ".format(rank,suit), end='')
        print("\n\tSoft score for this hand: ", self.soft_hand_score)
        print("\tHard score for this hand: ", self.hard_hand_score)
        print("\n\tBet on this hand: $", self.bet)
        if self.split_flag == True:
            print("\n\tSplit Hand: ", end='')
            for rank, suit in self.split_hand:
                print("{0}-{1}  ".format(rank,suit), end='')
            print("\n\tSoft score for this hand: ", self.soft_split_score)
            print("\tHard score for this hand: ", self.hard_split_score)
            print("\n\tBet on this hand: $", self.split_bet)
        print("\nInsurance against Dealer Blackjack: $", self.insurance)
        return "Data on "+ self.name + " is complete"
    
    def __del__(self):
        '''
        This method removes a player from the game. After deleting it, it prints a message
        and returns True.
        '''
        print("Player has been removed from the game.")
        return True
    
    def __len__(self):
        '''
        This method prints out the length of the player's regular hand. This is used to help
        determine a possible blackjack.
        '''
        return len(self.hand)
    
    def print_split(self):
        '''
        This method prints the player data in a format similar to __str__, but it leaves out
        the regular hand. It accepts no arguments.
        '''
        if self.split_flag == False:
            print("Player {0} does not have a split hand.".format(self.name))
            return False
        print("Player:\t", self.name)
        print("Chips:\t${0}.00".format(self.bank))
        print("\n\tSplit Hand: ", end='')
        for rank, suit in self.split_hand:
            print("{0}-{1}  ".format(rank,suit), end='')
        print("\n\tSoft score for this hand: ", self.soft_split_score)
        print("\tHard score for this hand: ", self.hard_split_score)
        print("\n\tBet on this hand: $", self.split_bet)
        return
    
    def add_card_to_hand(self, card):
        '''
        This method accepts a card tuple (rank, suit) as argument. It places this card into
        the Player.hand list. As long as at least one score for the hand is less than or 
        equal to 21, this method will return True. If both are greater than 21, it returns
        False, indicating a bust.
        
        Remember, soft scores treate Aces as 1, hard scores treat Aces as 11 unless that 
        would cause a bust. In that case, and all others, hard score and soft score are 
        the same.
        '''
        self.hand.append(card)
        rank,suit = card
        score = Player.values[rank]
        # The soft score will always be lower score because values treats Aces as a 1
        # point card. We increment both in case an each score in case an Ace occurred earlier
        # in the hand.
        self.soft_hand_score += score
        self.hard_hand_score += score
        
        # Now, we need to see if the soft score was a bust. If so, the player loses the hand
        # and the method returns False.
        if self.soft_hand_score > 21:
            return False
        # Now, we consider the case of this card being an Ace.
        if rank == 'A':
            test_score = self.hard_hand_score + 10
            print(test_score)
            # Checking to see if both scores for this Ace result in a playable hand. If so,
            # we need to increase the hard score by 10.
            if test_score <= 21:
                self.hard_hand_score = test_score
                return True
            # The hard score may still be greater than the soft score, but the hard score
            # cannot treat the Ace as an 11 without a bust. This is still a playable hand,
            # because we checked the soft score for a bust already.
        
        # Now, we need to consider the possibility that hard score was different due to an
        # earlier Ace. We would know that because soft != hard.
        if self.soft_hand_score != self.hard_hand_score and self.hard_hand_score > 21:
            self.hard_hand_score = self.soft_hand_score
        return True
    
    def add_card_to_split(self, card):
        '''
        This method accepts a card tuple (rank, suit) as argument. It places this card into
        the Player.split_hand list. As long as at least one score for the hand is less than or 
        equal to 21, this method will return True. If both are greater than 21, it returns
        False, indicating a bust.
        
        Remember, soft scores treate Aces as 1, hard scores treat Aces as 11 unless that 
        would cause a bust. In that case, and all others, hard score and soft score are 
        the same.
        
        '''
        if self.split_flag != True:
            print("Error: There is no split hand to add cards to.")
            return True
        
        self.split_hand.append(card)
        rank,suit = card
        score = Player.values[rank]
        # The soft score will always be lower score because values treats Aces as a 1
        # point card. We increment both in case an each score in case an Ace occurred earlier
        # in the hand.
        self.soft_split_score += score
        self.hard_split_score += score
        
        # Now, we need to see if the soft score was a bust. If so, the player loses the hand
        # and the method returns False.
        if self.soft_split_score > 21:
            return False
        # Now, we consider the case of this card being an Ace.
        if rank == 'A':
            test_score = self.hard_split_score + 10
            print(test_score)
            # Checking to see if both scores for this Ace result in a playable hand. If so,
            # we need to increase the hard score by 10.
            if test_score <= 21:
                self.hard_split_score = test_score
                return True
            # The hard score may still be greater than the soft score, but the hard score
            # cannot treat the Ace as an 11 without a bust. This is still a playable hand,
            # because we checked the soft score for a bust already.
        
        # Now, we need to consider the possibility that hard score was different due to an
        # earlier Ace. We would know that because soft != hard.
        if self.soft_split_score != self.hard_split_score and self.hard_split_score > 21:
            self.hard_split_score = self.soft_split_score
        return True
    
    def blackjack(self, multiplier):
        '''
        This method cleans up after a player achieved a blackjack (natural 21 on the first
        two cards dealt). Casinos always has a better payout ratio for a player winning a
        blackjack (assuming the Dealer doesn't tie with the player).
        
        This method does not return a value. The multiplier needs to be a decimal or an
        integer, not a fraction.
        
        Nothing is done to the split result because no split can occur with a natural 21.
        '''
        winnings = int(multiplier * self.bet)
        self.bank += winnings
        self.bet = 0
        self.hand = []
        self.soft_hand_score = self.hard_hand_score = 0
        return
    
    def win(self):
        '''
        This method cleans up after a player wins with their regular hand. The split_hand
        has a separate method for this purpose.
        
        This method does not return a value.
        '''
        # There is no multiplier for a regular win.
        self.bank += self.bet
        self.bet = 0
        self.hand = []
        self.soft_hand_score = self.hard_hand_score = 0
        return
    
    def split_win(self):
        '''
        This method cleans up a split hand after the split hand wins. The regular hand is dealt
        with in other methods.
        
        This method does not return a value.
        '''
        self.bank += self.split_bet
        self.split_bet = 0
        self.split_hand = []
        self.soft_split_score = self.hard_split_score = 0
        return

In [137]:
# player1 = Player("Fred")
player1.split_flag = True

player1.bet = 30
player1.hand = [('4','C'),('A', 'S'),('5', 'D')]
player1.soft_hand_score = 20
player1.hard_hand_score = 10
player1.split_flag = True
player1.split_hand = [('A', 'C'), ('10', 'H')]
player1.soft_split_score = 21
player1.hard_split_score = 11
player1.split_bet = 40

In [138]:
player1.split_bet = 50
print(player1.add_card_to_split(('4','C')))
# print(player1)
print(player1.add_card_to_split(('A', 'S')))
# print(player1)
print(player1.add_card_to_split(('5', 'D')))
# print(player1)
print(player1.add_card_to_split(('K', 'D')))
print(player1)

True
15
True
True
True
Player:	 Fred
Chips:	$1100.00

	Current Hand: 
	Soft score for this hand:  0
	Hard score for this hand:  0

	Bet on this hand: $ 0

	Split Hand: 4-C  A-S  5-D  K-D  
	Soft score for this hand:  20
	Hard score for this hand:  20

	Bet on this hand: $ 50

Insurance against Dealer Blackjack: $ 0
Data on Fred is complete


In [139]:
player1.split_win()
print(player1)

Player:	 Fred
Chips:	$1150.00

	Current Hand: 
	Soft score for this hand:  0
	Hard score for this hand:  0

	Bet on this hand: $ 0

	Split Hand: 
	Soft score for this hand:  0
	Hard score for this hand:  0

	Bet on this hand: $ 0

Insurance against Dealer Blackjack: $ 0
Data on Fred is complete


In [140]:
# del(player1)